### Test Module 3

### Download Chromosome 1 genome

In [ ]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta

--2022-07-04 19:46:54--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/chr1.GRCh38.excerpt.fasta
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.65.227.145, 18.65.227.2, 18.65.227.179, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.65.227.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 810105 (791K) [application/octet-stream]
Saving to: ‘chr1.GRCh38.excerpt.fasta’

chr1.GRCh38.excerpt 100%[===================>] 791.12K  2.29MB/s    in 0.3s    

2022-07-04 19:46:55 (2.29 MB/s) - ‘chr1.GRCh38.excerpt.fasta’ saved [810105/810105]



### Read Genome Function

In [ ]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

### Read fasta file (downloaded Human Chromosome 1 (genome))

In [ ]:
genome_file = 'chr1.GRCh38.excerpt.fasta'

t = readGenome(genome_file)

print(t[:100])

TTGAATGCTGAAATCAGCAGGTAATATATGATAATAGAGAAAGCTATCCCGAAGGTGCATAGGTCAACAATACTTGAGCCTAACTCAGTAGATCCTAAAA


### Original Edit Distance Function

In [ ]:
def editDistance(x, y):
    # Create distance matrix    
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return D[-1][-1]



### Modified Edit distance Function

In [ ]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i 
    for i in range(len(y)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1])

### Testing Modified Function using example in lecture

In [ ]:
 p ='GCGTATGC' 
 t ='TATTGGCTATACGGTT'

 print(editDistance(p,t))

2


### Question 1
##### What is the edit distance of the best match between pattern 
##### 'GCTGATCGATCGTACG' and the excerpt of Human Chromosome 1

In [ ]:
%%time
# Question1
p = 'GCTGATCGATCGTACG'

print(editDistance(p,t))

3
CPU times: user 9.35 s, sys: 13.8 ms, total: 9.36 s
Wall time: 9.38 s


### Question 2
##### What is the edit distance of the best match between pattern 
##### 'GATTTACCAGATTGAG' and the excerpt of Human Chromosome 1

In [ ]:
%%time
# Question2
p = 'GATTTACCAGATTGAG'

print(editDistance(p,t))

2
CPU times: user 9.44 s, sys: 23.6 ms, total: 9.46 s
Wall time: 9.49 s


### ReadFastq, Overlap, Smart_Overlap_Map Functions

In [ ]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities


def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match


def smart_overlap_map(reads, k):
    olaps = {}
    result = {}
    for read in reads:
        for i in range(len(read)-k+1):
            if read[i:i+k] not in olaps:
                olaps[read[i:i+k]] = [read]
            else:
                olaps[read[i:i+k]].append(read)

    count = 0
    for read in reads:
        read_suffix = read[-k:]
        for possible_read in olaps[read_suffix]:
            if possible_read != read:
                olen = overlap(read, possible_read, k)
                if olen > 0:
                    count += 1
                    result[(read, possible_read)] = olen

    return result, count

##### Find all pairs of reads with the exact suffix/prefix match of length 30.
##### Don't overlap a read with itself; if a read has a suffix.prefix match to
##### itself, ignore the match. Picture the overlap graph corresponding to the
##### overlaps just calculated

### Download Fastq file

In [ ]:
!wget --no-check https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq

--2022-07-04 21:12:31--  https://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.for_asm.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 18.65.227.145, 18.65.227.2, 18.65.227.179, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|18.65.227.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2562951 (2.4M) [application/octet-stream]
Saving to: ‘ERR266411_1.for_asm.fastq’

ERR266411_1.for_asm 100%[===================>]   2.44M  5.09MB/s    in 0.5s    

2022-07-04 21:12:32 (5.09 MB/s) - ‘ERR266411_1.for_asm.fastq’ saved [2562951/2562951]



### Read Fastq File

In [ ]:
reads_filename = 'ERR266411_1.for_asm.fastq'
reads, _ = readFastq(reads_filename)

### Question 3
##### How many edges are in the graph?
##### In other words, how many distinct pairs of reads overlap?

In [ ]:
pairs_olen, pairs_count = smart_overlap_map(reads, 30)

# Question 3
print("Number of pairs of reads overlap:", pairs_count)

Number of pairs of reads overlap: 904746


### Question 4
##### How many reads have a suffix involved

In [ ]:
# Question 4
reads_involved = []
for key, value in pairs_olen:
    reads_involved.append(key)
print("Number of reads have a suffix involved in an overlap:", len(set(reads_involved)))

Number of reads have a suffix involved in an overlap: 7161
